In [1]:
import sys, os, glob
sys.path.append('../')

import tensorflow as tf
from tensorflow.keras.models import model_from_json

from MRIsegm.metrics import dice_coef
from segmentation_models.losses import DiceLoss, BinaryFocalLoss

Segmentation Models: using `keras` framework.


In [3]:
models_dir = '../data/models'
models_path = glob.glob(models_dir + '/*full_150E_O*focal*.h5')

dice_loss = DiceLoss()
focal_loss = BinaryFocalLoss()
loss = dice_loss + (1 * focal_loss)

dependencies = {
    'dice_coef': dice_coef,
    'FixedDropout': tf.keras.layers.Dropout(0.2),
    'dice_loss': dice_loss,
    'dice_loss_plus_1binary_focal_loss': loss

}

In [4]:
models_name = [ os.path.splitext(os.path.split(models_path[i])[1])[0] for i, mod in enumerate(models_path)]
models_loaded = [ tf.keras.models.load_model(models_path[i], custom_objects=dependencies) for i, mod in enumerate(models_path)]

In [5]:
import json
weights_dir = '../data/models/weights'

for k, mod in enumerate(models_loaded):

    model_architecture = mod.to_json()

    with open(weights_dir + '/' + models_name[k] + ".json", "w") as file:
        json.dump(model_architecture, file)

    output = weights_dir + '/' + models_name[k] + '_weights.h5'
    mod.save_weights(output)

/Users/giuseppefilitto/miniforge3/envs/tf2.5/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
